# 加密市場數據探索

本notebook展示如何：
1. 連接TimescaleDB
2. 讀取OHLCV數據
3. 基本統計分析
4. 視覺化價格走勢
5. 計算技術指標

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os

# 設定視覺化風格
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. 連接資料庫

In [ ]:
# 從環境變數讀取資料庫配置
DB_HOST = os.getenv('POSTGRES_HOST', 'localhost')
DB_PORT = os.getenv('POSTGRES_PORT', '5432')
DB_USER = os.getenv('POSTGRES_USER', 'crypto')
DB_PASS = os.getenv('POSTGRES_PASSWORD', 'crypto_pass')
DB_NAME = os.getenv('POSTGRES_DB', 'crypto_db')

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

print("Database connected successfully!")

## 2. 讀取OHLCV數據

In [ ]:
# 查詢最近7天的1分鐘K線數據
query = """
SELECT 
    o.open_time,
    o.open,
    o.high,
    o.low,
    o.close,
    o.volume,
    m.symbol
FROM ohlcv o
JOIN markets m ON o.market_id = m.id
WHERE o.timeframe = '1m'
  AND o.open_time >= NOW() - INTERVAL '7 days'
ORDER BY o.open_time ASC
"""

df = pd.read_sql(query, engine, parse_dates=['open_time'])
df.set_index('open_time', inplace=True)

print(f"Loaded {len(df)} candles")
df.head()

## 3. 基本統計分析

In [ ]:
# 計算回報率
df['returns'] = df['close'].pct_change()
df['log_returns'] = np.log(df['close'] / df['close'].shift(1))

# 統計摘要
print("=== Price Statistics ===")
print(df[['open', 'high', 'low', 'close', 'volume']].describe())

print("\n=== Returns Statistics ===")
print(f"Mean Return: {df['returns'].mean():.6f}")
print(f"Volatility (std): {df['returns'].std():.6f}")
print(f"Sharpe Ratio (annualized): {(df['returns'].mean() / df['returns'].std()) * np.sqrt(525600):.2f}")

## 4. 視覺化價格走勢

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# 收盤價
axes[0].plot(df.index, df['close'], label='Close Price', linewidth=1)
axes[0].set_title('BTC/USDT Close Price', fontsize=14)
axes[0].set_ylabel('Price (USDT)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 成交量
axes[1].bar(df.index, df['volume'], alpha=0.5, label='Volume')
axes[1].set_ylabel('Volume', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 回報率分布
axes[2].hist(df['returns'].dropna(), bins=100, alpha=0.7, edgecolor='black')
axes[2].set_title('Returns Distribution', fontsize=14)
axes[2].set_xlabel('Returns', fontsize=12)
axes[2].set_ylabel('Frequency', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 計算技術指標

In [ ]:
# 移動平均線
df['ma_20'] = df['close'].rolling(window=20).mean()
df['ma_50'] = df['close'].rolling(window=50).mean()

# 布林通道
df['bb_mid'] = df['close'].rolling(window=20).mean()
df['bb_std'] = df['close'].rolling(window=20).std()
df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']

# RSI
delta = df['close'].diff()
gain = delta.where(delta > 0, 0).rolling(window=14).mean()
loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
rs = gain / loss
df['rsi'] = 100 - (100 / (1 + rs))

# 視覺化
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 價格 + MA + Bollinger Bands
axes[0].plot(df.index, df['close'], label='Close', linewidth=1.5)
axes[0].plot(df.index, df['ma_20'], label='MA20', linestyle='--', alpha=0.7)
axes[0].plot(df.index, df['ma_50'], label='MA50', linestyle='--', alpha=0.7)
axes[0].fill_between(df.index, df['bb_upper'], df['bb_lower'], alpha=0.2, label='BB')
axes[0].set_title('Price with Technical Indicators', fontsize=14)
axes[0].set_ylabel('Price (USDT)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(df.index, df['rsi'], label='RSI', color='purple', linewidth=1.5)
axes[1].axhline(70, color='r', linestyle='--', alpha=0.5, label='Overbought')
axes[1].axhline(30, color='g', linestyle='--', alpha=0.5, label='Oversold')
axes[1].set_title('Relative Strength Index (RSI)', fontsize=14)
axes[1].set_ylabel('RSI', fontsize=12)
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 簡單價格方向預測（Baseline）

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 特徵工程
df['price_change'] = df['close'].pct_change()
df['volume_change'] = df['volume'].pct_change()
df['high_low_ratio'] = df['high'] / df['low']
df['target'] = (df['close'].shift(-1) > df['close']).astype(int)  # 下一根K線是否上漲

# 選擇特徵
features = ['price_change', 'volume_change', 'high_low_ratio', 'rsi', 'ma_20', 'ma_50']
df_ml = df[features + ['target']].dropna()

X = df_ml[features]
y = df_ml['target']

# 分割訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# 訓練隨機森林分類器
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 預測與評估
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n=== Model Performance ===")
print(f"Accuracy: {accuracy:.4f}")
print(f"\n{classification_report(y_test, y_pred, target_names=['Down', 'Up'])}")

# 特徵重要性
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== Feature Importance ===")
print(feature_importance)

## 總結

本notebook完成了：
1. ✅ 資料庫連接與數據讀取
2. ✅ 基本統計分析
3. ✅ 價格與成交量視覺化
4. ✅ 技術指標計算（MA、BB、RSI）
5. ✅ 簡單機器學習模型（價格方向預測）

### 下一步方向：
- 加入更多技術指標（MACD、KDJ等）
- 使用深度學習模型（LSTM、Transformer）
- 實作完整的回測框架
- 加入風險管理模組